In [ ]:
import numpy as np
import pandas as pd

### GFOP sample type metadata 

In [ ]:
def get_sample_types(gfop_metadata, simple_complex=None):
    if simple_complex is not None:
        gfop_metadata = gfop_metadata[
            gfop_metadata['simple_complex'] == simple_complex]
    col_sample_types = [f'sample_type_group{i}' for i in range(1, 7)]
    return (gfop_metadata[['filename', *col_sample_types]]
            .set_index('filename'))

In [ ]:
gfop_metadata = pd.read_csv(
    '../data/11442_foodomics_multiproject_metadata_20200630_submission.txt', sep='\t')
gfop_metadata = gfop_metadata.apply(lambda col: col.str.strip()
                                    if col.dtype == 'object' else col)

In [ ]:
gfop_metadata.head()

### Food count per file

In [ ]:
def get_file_food_counts(gnps_network, sample_types, all_groups, some_groups,
                         filenames_included, level):
    # Select GNPS job groups.
    groups = {f'G{i}' for i in range(1, 7)}
    groups_excluded = groups - set([*all_groups, *some_groups])
    df_selected = gnps_network[
        (gnps_network[all_groups] > 0).all(axis=1) &
        (gnps_network[some_groups] > 0).any(axis=1) &
        (gnps_network[groups_excluded] == 0).all(axis=1)].copy()
    df_selected = df_selected[
        df_selected['UniqueFileSources'].apply(lambda cluster_fn:
            any(fn in cluster_fn for fn in filenames_included))]
    filenames = (df_selected['UniqueFileSources'].str.split('|')
                 .explode())
    # Select food hierarchy levels.
    sample_types = sample_types[f'sample_type_group{level}'] # remove this line to do file-level
    # Match the GNPS job results to the food sample types.
    sample_types_selected = sample_types.reindex(filenames)
    sample_types_selected = sample_types_selected.dropna()
    # Discard samples that occur less frequent than water (blank).
    water_count = (sample_types_selected == 'water').sum()
    sample_counts = sample_types_selected.value_counts()
    sample_counts_valid = sample_counts.index[sample_counts > water_count]
    sample_types_selected = sample_types_selected[
        sample_types_selected.isin(sample_counts_valid)]
    # Get sample counts at the specified level.
    return sample_types_selected.value_counts()

In [ ]:
sample_types_simple = get_sample_types(gfop_metadata, 'simple')
sample_types_complex = get_sample_types(gfop_metadata, 'complex')

In [ ]:
metadata = pd.read_csv(
    '../data/QiitaID_11546_Metadata_041420_JMG_matched.txt', sep='\t')
filename_col = 'filename'

In [ ]:
gnps_network = pd.read_csv(
    '../data/METABOLOMICS-SNETS-V2-07f85565-view_all_clusters_withID_beta-main.tsv',
    sep='\t')

In [ ]:
level = 6
food_counts, filenames = [], []
        
#Explicit selection of categories.
some_groups = ['G4']
all_groups = ['G1']
for filename in metadata[filename_col]:
    file_food_counts = get_file_food_counts(
        gnps_network, sample_types_simple, all_groups, some_groups, [filename], level)
    if len(file_food_counts) > 0:
        food_counts.append(file_food_counts)
        filenames.append(filename)

In [ ]:
food_counts = (pd.concat(food_counts, axis=1, sort=True)
               .fillna(0).astype(int).T)
food_counts.index = pd.Index(filenames, name='filename')

In [ ]:
food_counts.to_csv('food_counts_L6.csv')